In [4]:
%pip install smolagents python-dotenv sqlalchemy --upgrade -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
import os

# Cargar las variables de entorno del archivo .env
load_dotenv()

True

In [3]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

In [4]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


In [5]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

In [6]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel("meta-llama/Meta-Llama-3.1-8B-Instruct"),
)
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Meta-Llama-3.1-8B-Instruct ────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  receipts = sql_engine(query="SELECT customer_name, price FROM receipts")                                         
  print("List of receipts and their corresponding prices from the \"receipts\" table:")                            
  print(receipts)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
List of receipts and their corresponding prices from the "receipts" table:

('Alan Payne', 12.06)
('Alex Mason', 23.86)
('Woodrow Wilson', 53.43)
('Margaret James', 21.11)

Out: None

[Step 0: Duration 3.33 seconds| Input tokens: 2,109 | Output tokens: 91]

...

Code execution failed at line 'for receipt in receipts:
    if receipt[1] == max_price:
        final_answer(receipt[0])
        break' due to: InterpreterError: Index 1 out of bounds for string of length 1

[Step 5: Duration 3.75 seconds| Input tokens: 16,584 | Output tokens: 615]

Reached max steps.

[Step 6: Duration 3.75 seconds| Input tokens: 18,649 | Output tokens: 851]

'Thought: I need to get the maximum price, then use a tool to get the corresponding customer name from the table.\n\nCode:\n```py\n müşteri the maximum price \nmax_price = sql_engine(query="SELECT MAX(price) FROM receipts")\nprint("The maximum price:")\nprint(max_price)\nös for ığısı to retrieve all \nreceipts = sql_engine(query="SELECT customer_name, price FROM receipts")\nprint("The list of receipts:")\nimport operator\nprint(receipts)\nmax_price_value = float(max_price[0])\nmax_price_result = max(receipts, key=operator.itemgetter(1))\nfinal_answer(max_price_result[0])\n```\n\nAfter the code, I will get output:\npython\nmax_price = sql_engine(query="SELECT MAX(price) FROM receipts")\nprint("The maximum price:")\nprint(max_price) \nreceipts = sql_engine(query="SELECT customer_name, price FROM receipts")\nprint("The list of receipts:")\nprint(receipts)\nimport operator\nmax_price_value = float(max_price[0])\nmax_price_result = max(receipts, key=operator.itemgetter(1))\nfinal_answer(max